In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore') # Es bueno para ocultar advertencias futuras

# --- Cargar el 'banco de datos' ---
# Asegúrate de que tu archivo 'loan.csv' esté en la misma carpeta que este notebook
try:
    df = pd.read_csv('loan.csv')

    print("¡Éxito! Archivo 'loan.csv' cargado.")
    print("-" * 50)

    # 1. Imprimir la cantidad de filas y columnas
    print(f"El dataset tiene: {df.shape[0]} filas")
    print(f"El dataset tiene: {df.shape[1]} columnas")
    print("-" * 50)

    # 2. Imprimir la lista de nombres de columnas
    print("Los nombres de las columnas son:")
    # (Usamos .tolist() para que se imprima la lista completa y no la corte)
    print(df.columns.tolist())
    print("-" * 50)

    # 3. Imprimir el resumen (MUY útil)
    # Esto nos dirá los tipos de datos (Dtype) y cuántos valores nulos hay
    print("Resumen del DataFrame (df.info()):")
    df.info()
    
    # 4. (Opcional) Ver las primeras 5 filas para darnos una idea
    print("\nPrimeras 5 filas del dataset:")
    pd.set_option('display.max_columns', None) # Para que muestre TODAS las columnas
    print(df.head())


except FileNotFoundError:
    print("¡ERROR! No se encontró el archivo 'loan.csv'.")
    print("Asegúrate de que el archivo esté en la misma carpeta que tu notebook.")
except Exception as e:
    print(f"Ocurrió un error al cargar el archivo: {e}")

¡Éxito! Archivo 'loan.csv' cargado.
--------------------------------------------------
El dataset tiene: 2260668 filas
El dataset tiene: 145 columnas
--------------------------------------------------
Los nombres de las columnas son:
['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_m

In [2]:
# Contemos cuántos préstamos hay de cada tipo
print("Valores en 'loan_status':")
print(df['loan_status'].value_counts())

Valores en 'loan_status':
loan_status
Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: count, dtype: int64


In [3]:
# Filtramos el DataFrame.
# Esto reducirá tus 2.2M de filas a un número más manejable (quizás 1.3M)
df_filtrado = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]

print(f"\nFilas antes del filtro: {df.shape[0]}")
print(f"Filas después del filtro: {df_filtrado.shape[0]}")

# Ahora, creamos nuestro objetivo (Target)
# 1 = No Pagó (Charged Off)
# 0 = Pagó (Fully Paid)
df_filtrado['incumplimiento'] = df_filtrado['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)


Filas antes del filtro: 2260668
Filas después del filtro: 1303607


In [5]:
# Esta es nuestra "lista de deseos" de columnas clave.
# Todas estas son cosas que le preguntas a un cliente en el formulario.
columnas_clave = [
    'loan_amnt',         # Monto que pide
    'term',              # Plazo (36 o 60 meses)
    'int_rate',          # Tasa de interés (esta se la *asigna* el banco)
    'grade',             # Calificación (A, B, C...)
    'home_ownership',    # Tipo de vivienda (RENT, OWN, MORTGAGE)
    'annual_inc',        # Ingreso anual
    'verification_status', # Si el ingreso fue verificado
    'purpose',           # Propósito (debt_consolidation, car, etc.)
    'dti',               # Ratio Deuda/Ingreso (¡el que te expliqué!)
    'open_acc',          # Cuántas cuentas de crédito tiene abiertas
    'pub_rec_bankruptcies', # Si ha tenido bancarrotas públicas
    'incumplimiento'     # Nuestro objetivo (0 o 1)
]

df_seleccionado = df_filtrado[columnas_clave]

print("\nDataset con columnas seleccionadas:")
df_seleccionado.info()


Dataset con columnas seleccionadas:
<class 'pandas.core.frame.DataFrame'>
Index: 1303607 entries, 100 to 2260664
Data columns (total 12 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   loan_amnt             1303607 non-null  int64  
 1   term                  1303607 non-null  object 
 2   int_rate              1303607 non-null  float64
 3   grade                 1303607 non-null  object 
 4   home_ownership        1303607 non-null  object 
 5   annual_inc            1303607 non-null  float64
 6   verification_status   1303607 non-null  object 
 7   purpose               1303607 non-null  object 
 8   dti                   1303295 non-null  float64
 9   open_acc              1303607 non-null  float64
 10  pub_rec_bankruptcies  1302910 non-null  float64
 11  incumplimiento        1303607 non-null  int64  
dtypes: float64(5), int64(2), object(5)
memory usage: 129.3+ MB


In [6]:
# 'dti' a veces tiene nulos. Los llenaremos con 0 (un valor conservador).
df_seleccionado['dti'] = df_seleccionado['dti'].fillna(0)

# 'pub_rec_bankruptcies' a veces tiene nulos. Llenar con 0.
df_seleccionado['pub_rec_bankruptcies'] = df_seleccionado['pub_rec_bankruptcies'].fillna(0)

# Y eliminamos cualquier otra fila que AÚN tenga nulos (serán muy pocas)
df_limpio = df_seleccionado.dropna()

print("\nDataset limpio y listo para procesar:")
df_limpio.info()


Dataset limpio y listo para procesar:
<class 'pandas.core.frame.DataFrame'>
Index: 1303607 entries, 100 to 2260664
Data columns (total 12 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   loan_amnt             1303607 non-null  int64  
 1   term                  1303607 non-null  object 
 2   int_rate              1303607 non-null  float64
 3   grade                 1303607 non-null  object 
 4   home_ownership        1303607 non-null  object 
 5   annual_inc            1303607 non-null  float64
 6   verification_status   1303607 non-null  object 
 7   purpose               1303607 non-null  object 
 8   dti                   1303607 non-null  float64
 9   open_acc              1303607 non-null  float64
 10  pub_rec_bankruptcies  1303607 non-null  float64
 11  incumplimiento        1303607 non-null  int64  
dtypes: float64(5), int64(2), object(5)
memory usage: 129.3+ MB


In [7]:
# 'term' tiene " 36 months". Quitamos el texto.
df_limpio['term'] = df_limpio['term'].apply(lambda x: int(x.split()[0]))

# 'grade' es A, B, C... lo convertimos a números (A=0, B=1...)
# (No te preocupes por este código, solo cópialo, es para ordenar las letras)
from pandas.api.types import CategoricalDtype
grade_ordenado = CategoricalDtype(categories=['A', 'B', 'C', 'D', 'E', 'F', 'G'], ordered=True)
df_limpio['grade'] = df_limpio['grade'].astype(grade_ordenado).cat.codes

# Ahora, convertimos 'home_ownership', 'verification_status' y 'purpose'
df_final = pd.get_dummies(df_limpio, 
                          columns=['home_ownership', 'verification_status', 'purpose'], 
                          drop_first=True) # drop_first=True es importante

print("\n--- ¡DATASET FINAL LISTO PARA ENTRENAR! ---")
print(df_final.head())


--- ¡DATASET FINAL LISTO PARA ENTRENAR! ---
     loan_amnt  term  int_rate  grade  annual_inc    dti  open_acc  \
100      30000    36     22.35      3    100000.0  30.46      11.0   
152      40000    60     16.14      2     45000.0  50.53      18.0   
170      20000    36      7.56      0    100000.0  18.92       9.0   
186       4500    36     11.31      1     38500.0   4.64      12.0   
215       8425    36     27.27      4    450000.0  12.37      21.0   

     pub_rec_bankruptcies  incumplimiento  home_ownership_MORTGAGE  \
100                   1.0               0                     True   
152                   0.0               0                     True   
170                   0.0               0                     True   
186                   0.0               0                    False   
215                   0.0               0                     True   

     home_ownership_NONE  home_ownership_OTHER  home_ownership_OWN  \
100                False                 Fa

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# X son TODAS las columnas MENOS nuestro objetivo 'incumplimiento'
# df_final es el DataFrame que acabamos de crear
X = df_final.drop('incumplimiento', axis=1)

# y es SOLAMENTE la columna 'incumplimiento' (0 o 1)
y = df_final['incumplimiento']

print(f"Forma de X (las variables de decisión): {X.shape}")
print(f"Forma de y (el objetivo a predecir): {y.shape}")

# Dividir los datos (70% para entrenar, 30% para probar)
# Usamos un 30% para 'test_size' porque tenemos muchísimos datos.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Forma de X (las variables de decisión): (1303607, 28)
Forma de y (el objetivo a predecir): (1303607,)


In [11]:
# 1. Inicializar el modelo
# n_estimators=100 significa que construirá 100 "árboles de decisión"
# n_jobs=-1 usa todos los núcleos de tu CPU para acelerar
model = RandomForestClassifier(n_estimators=100, 
                             random_state=42, 
                             n_jobs=-1, 
                             verbose=1,
                             class_weight='balanced')
# 'verbose=1' mostrará texto mientras entrena, para que veas que no se trabó

# 2. Entrenar el modelo
print("--- INICIANDO ENTRENAMIENTO ---")
print(f"Alimentando el modelo con {X_train.shape[0]} filas de datos...")
print("Esto tardará varios minutos (10-30 min)...")

model.fit(X_train, y_train) # ¡Aquí ocurre la magia!

print("--- ¡ENTRENAMIENTO COMPLETADO! ---")

--- INICIANDO ENTRENAMIENTO ---
Alimentando el modelo con 912524 filas de datos...
Esto tardará varios minutos (10-30 min)...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.


--- ¡ENTRENAMIENTO COMPLETADO! ---


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   15.0s finished


In [12]:
# 3. Evaluar el modelo con los datos de prueba (el 30% que nunca vio)
print("Evaluando el modelo...")
y_pred = model.predict(X_test)

print(f"\nPrecisión del modelo: {accuracy_score(y_test, y_pred) * 100:.2f}%")

# El Reporte de Clasificación es CLAVE
# Te dirá qué tan bueno es prediciendo '0' (Pagó) vs '1' (No Pagó)
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred, target_names=['Pagó (0)', 'No Pagó (1)']))

# 4. Guardar el "cerebro" (el modelo) y las columnas
import joblib

print("\nGuardando el modelo en 'modelo_prestamos.pkl'...")
joblib.dump(model, 'modelo_prestamos.pkl')

# También guarda la lista de columnas.
# ¡Esto es CRÍTICO para que la app de Streamlit funcione!
columnas_modelo = X.columns.tolist()
joblib.dump(columnas_modelo, 'columnas_modelo.pkl')

print("¡Modelo y columnas guardados con éxito!")

Evaluando el modelo...


[Parallel(n_jobs=32)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done 100 out of 100 | elapsed:    1.5s finished



Precisión del modelo: 79.69%

Reporte de Clasificación:
              precision    recall  f1-score   support

    Pagó (0)       0.81      0.98      0.88    312239
 No Pagó (1)       0.48      0.08      0.14     78844

    accuracy                           0.80    391083
   macro avg       0.64      0.53      0.51    391083
weighted avg       0.74      0.80      0.74    391083


Guardando el modelo en 'modelo_prestamos.pkl'...
¡Modelo y columnas guardados con éxito!


In [13]:
!pip install lightgbm
!pip install imbalanced-learn


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\allan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\allan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from lightgbm import LGBMClassifier # ¡El nuevo modelo!
import joblib

# 1. DEFINIR X e Y (igual que antes)
X = df_final.drop('incumplimiento', axis=1)
y = df_final['incumplimiento']

# 2. DIVIDIR DATOS (igual que antes)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. ¡LA MAGIA! APLICAR SUB-MUESTREO (Undersampling)
# Solo lo aplicamos a los datos de ENTRENAMIENTO (X_train, y_train)
print("--- APLICANDO SUB-MUESTREO ---")
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

print(f"Filas de Entrenamiento (Antes): {len(y_train)}")
print(f"Filas de Entrenamiento (Balanceado): {len(y_train_res)}")
print("Nuevos valores de 'y_train_res':")
print(y_train_res.value_counts()) # ¡Ahora debería estar 50/50!

# 4. ENTRENAR EL NUEVO MODELO (LGBM)
# LGBM es más rápido e inteligente para esto que RandomForest
print("\n--- INICIANDO ENTRENAMIENTO (con LGBM y datos balanceados) ---")
# n_estimators=200 le da más poder de aprendizaje
model = LGBMClassifier(n_estimators=200, random_state=42, n_jobs=-1)

model.fit(X_train_res, y_train_res) # ¡Entrenamos con los datos balanceados!

print("--- ¡ENTRENAMIENTO COMPLETADO! ---")

# 5. EVALUAR EL NUEVO MODELO
# Evaluamos en el X_test original (¡el que no está balanceado!)
print("Evaluando el nuevo modelo...")
y_pred = model.predict(X_test)

print(f"\nPrecisión del modelo (NUEVA): {accuracy_score(y_test, y_pred) * 100:.2f}%")

print("\nReporte de Clasificación (NUEVO):")
print(classification_report(y_test, y_pred, target_names=['Pagó (0)', 'No Pagó (1)']))

# 6. GUARDAR EL MODELO BUENO
print("\nGuardando el modelo en 'modelo_prestamos.pkl'...")
joblib.dump(model, 'modelo_prestamos.pkl')

# Guardar las columnas (son las mismas de antes)
columnas_modelo = X.columns.tolist()
joblib.dump(columnas_modelo, 'columnas_modelo.pkl')

print("¡Modelo y columnas guardados con éxito!")

--- APLICANDO SUB-MUESTREO ---
Filas de Entrenamiento (Antes): 912524
Filas de Entrenamiento (Balanceado): 365622
Nuevos valores de 'y_train_res':
incumplimiento
0    182811
1    182811
Name: count, dtype: int64

--- INICIANDO ENTRENAMIENTO (con LGBM y datos balanceados) ---
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 182811, number of negative: 182811
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005843 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1126
[LightGBM] [Info] Number of data points in the train set: 365622, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
--- ¡ENTRENAMIENTO COMPLETADO! ---
Evaluando el nuevo modelo...

Precisión del modelo (NUEVA): 63.87%

Reporte de Clasificación (NUEVO):
 